In [1]:
import requests
import pandas as pd
import numpy as np 
from keys import * #Se importan los paquetes necesarios para poder leer y arrojar resultados en formatos optimos, en el caso de keys
                    #Se importa el archivo y se cambia su extensión a .py
city = "Montréal"
country = "CA"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
# Se ingresan los datos con los cuales se realizará la busqueda, ciudad, país y los datos que se extraerán de la página

In [2]:
data = response.json()# Convierte la respuesta de la API a formato JSON
print (data)# Imprime la respuesta

# Extract forecast list
forecast_list = data.get('list', [])# Extrae la lista de pronósticos desde el JSON.

# Prepara listas vacías para almacenar los datos que usaremos en el DataFrame
# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

#Recorremos cada dato de la lista
for entry in forecast_list:
    # Extrae la fecha y hora del pronóstico
    times.append(entry.get('dt_txt', np.nan))
    # Extrae la temperatura
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    # Extrae la humedad
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
     # Extrae el estado del clima
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    # Extrae la velocidad del viento
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    # Extrae la cantidad de lluvia en las últimas 3 horas
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    # Extrae la cantidad de nieve en las últimas 3 horas 
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame # Se crea un DataFrame con los datos recolectados
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"CA"
})
# Se muestran las primeras filas del DataFrame
print(df.head())

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1761868800, 'main': {'temp': 5.67, 'feels_like': 2.78, 'temp_min': 5.67, 'temp_max': 6.83, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 1002, 'humidity': 97, 'temp_kf': -1.16}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'clouds': {'all': 100}, 'wind': {'speed': 3.82, 'deg': 39, 'gust': 8.07}, 'visibility': 10000, 'pop': 1, 'rain': {'3h': 3.72}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-10-31 00:00:00'}, {'dt': 1761879600, 'main': {'temp': 6.39, 'feels_like': 3.48, 'temp_min': 6.39, 'temp_max': 7.82, 'pressure': 1004, 'sea_level': 1004, 'grnd_level': 998, 'humidity': 96, 'temp_kf': -1.43}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.17, 'deg': 60, 'gust': 10.1}, 'visibility': 10000, 'pop': 1, 'rain': {'3h': 1.52}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-10-31 03:00:00'}, {'dt': 1761890400, 'main': {'temp

In [3]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
from sqlalchemy import create_engine

user = "root"
password = "HKnight03%2F08%2F05"  # password con / codificado
host = "localhost"
port = 3306
schema = "gans"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")
con = engine.connect()
print("Conectado a MySQL")

Conectado a MySQL


In [4]:
df["municipality_iso_country"] = "CA"

In [6]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = engine, index=False)


40